In [2]:
!pip install -q mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 26.1 MB/s eta 0:00:00


In [6]:
import mediapipe
import cv2
from google.colab.patches import cv2_imshow
import math
import numpy as np
import os
"""
this file implements the process of storing world_pose_landmarks from images to csv.
"""

dataset_path = "/content/drive/MyDrive/deadlift_dataset/"
dataset_dict = {}


for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    if os.path.isdir(folder_path):
        images = []
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            if os.path.isfile(img_path):
                img = cv2.imread(img_path)
                images.append(img)
        dataset_dict[folder_name] = images


for key in dataset_dict:
  while len(dataset_dict[key]) % 3 !=0:
    dataset_dict[key].pop()
  print(f'the length of {key} folder is: {len(dataset_dict[key])}')

the length of 0(excellent) folder is: 3
the length of 1(back error) folder is: 0
the length of 2(foot error) folder is: 0
the length of 3(head error) folder is: 0


In [9]:
import mediapipe as mp
mp_pose = mp.solutions.pose
results = {}
with mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.5, model_complexity=1) as pose:
  for folder_name in dataset_dict.keys():
    results_by_folder = list()
    for image in dataset_dict[folder_name]:
      results_by_folder.append(pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))
    results[folder_name] = results_by_folder
results

{'0(excellent)': [mediapipe.python.solution_base.SolutionOutputs,
  mediapipe.python.solution_base.SolutionOutputs,
  mediapipe.python.solution_base.SolutionOutputs],
 '1(back error)': [],
 '2(foot error)': [],
 '3(head error)': []}

In [20]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset_for_deadlift.csv', nrows=0)
for folder_name in results:
  target = folder_name[:1]
  row = list()
  index = 0
  for result in results[folder_name]:
    if result.pose_world_landmarks.landmark:
      for lms in result.pose_world_landmarks.landmark:
        row.append(lms.x)
        row.append(lms.y)
        row.append(lms.z)
      index+=1
      if index==3:
        index = 0
        row.append(target)
        df.loc[len(df)] = row
        row.clear()

In [19]:
df

,(0_screen)x_0.nose,(0_screen)y_0.nose,(0_screen)z_0.nose,(0_screen)x_1.left_eye_inner,(0_screen)y_1.left_eye_inner,(0_screen)z_1.left_eye_inner,(0_screen)x_2.left_eye,(0_screen)y_2.left_eye,(0_screen)z_2.left_eye,(0_screen)x_3.left_eye_outer,...,(2_screen)x_30.right_heel,(2_screen)y_30.right_heel,(2_screen)z_30.right_heel,(2_screen)x_31.left_foot_index,(2_screen)y_31.left_foot_index,(2_screen)z_31.left_foot_index,(2_screen)x_32.right_foot_index,(2_screen)y_32.right_foot_index,(2_screen)z_32.right_foot_index,target
0,0.398293,-0.300328,-0.465071,0.397638,-0.346094,-0.447795,0.398017,-0.346218,-0.446967,0.398478,...,-0.093671,0.723405,-0.05968,0.282605,0.79464,0.067875,-0.058824,0.825656,-0.11694,0


In [ ]:
df.to_csv('/content/drive/MyDrive/dataset_for_deadlift.csv', index=False)